In [ ]:
!mamba install t-coffee mmseqs2 -y -q

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from learnMSA.msa_hmm import Configuration, Align, Visualize, Emitter, Transitioner, Initializers, Training
from learnMSA.msa_hmm.SequenceDataset import SequenceDataset
from learnMSA.protein_language_models import Common

2024-01-10 10:06:34.365444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 10:06:34.450866: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-10 10:06:34.869448: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/beckerf/mambaforge/envs/learnMSAdev/lib/:/home/beckerf/mambaforge/envs/learnMSAdev/lib/python3.10/site-packages/nvidia/cudnn/lib:
2024-01-10 10:06:34.869663: W tensorflow/stream_executor/platform/default/ds


# Experimental demo: learnMSA + protein language model

This notebook demonstrates how to align a set of protein sequences with learnMSA supported by a large, pre-trained protein language model.

This configuration of learnMSA - although a prototype - is the most accurate variant of learnMSA currently available. It is also the most computationally expensive. It is recommended to run this notebook on a GPU.

In [2]:
# Your fasta file with unaligned sequences.

#train_filename = "test/data/egf.fasta"
train_filename = "../../snakeMSA/data/homfam/train/rhv"

# Reference file with aligned sequences that have matching IDs to (potentially a subset of) the 
# sequences in the train_file.
# Replace with empty string if no reference is available.
#ref_filename = "test/data/egf.ref"
ref_filename = "../../snakeMSA/data/homfam/refs/rhv"

# The number of independently trained models.
num_models = 10

HMM training supported by protein embeddings:

In [3]:
def align(filename, out_filename):
    scoring_model_config = Common.ScoringModelConfig(dim=32, lm_name="protT5", activation="softmax", scaled=False)
    config = Configuration.make_default(num_models, 
                                        use_language_model=True, 
                                        scoring_model_config=scoring_model_config,
                                        frozen_insertions=True,
                                        num_prior_components=10,
                                        V2_emitter=True,
                                        V2_temperature=3.)
    # we have to define a special model- and batch generator if using a language model
    # because the emission probabilities are computed differently and the LM requires specific inputs
    model_gen = Training.make_generic_embedding_model_generator(config["scoring_model_config"].dim)
    batch_gen = Training.EmbeddingBatchGenerator(config["scoring_model_config"])
    with SequenceDataset(train_filename, fmt="fasta") as data:
        alignment_model = Align.run_learnMSA(data,
                                            out_filename,
                                            config, 
                                            model_generator=model_gen,
                                            batch_generator=batch_gen,
                                            sequence_weights=Align.compute_sequence_weights(train_filename, "tmp", config["cluster_seq_id"]),
                                            verbose=True,
                                            align_insertions=True)
    return alignment_model

In [4]:
alignment_model = align(train_filename, "test/data/interactive.alignment.fasta") 
Visualize.print_and_plot(alignment_model, alignment_model.best_model)

Training of 10 models on file rhv
Configuration: 
{
num_models : 10
transitioner : ProfileHMMTransitioner(
 transition_init=
    {
    begin_to_match : DefaultEntry() , match_to_end : DefaultExit() , 
    match_to_match : DefaultMatchTransition(1) , match_to_insert : DefaultMatchTransition(-1) , 
    insert_to_match : Norm(0, 0.1) , insert_to_insert : Norm(-0.5, 0.1) , 
    match_to_delete : DefaultMatchTransition(-1) , delete_to_match : Norm(0, 0.1) , 
    delete_to_delete : Norm(-0.5, 0.1) , left_flank_loop : Norm(0, 0.1) , 
    left_flank_exit : Norm(-1, 0.1) , right_flank_loop : Norm(0, 0.1) , 
    right_flank_exit : Norm(-1, 0.1) , unannotated_segment_loop : Norm(0, 0.1) , 
    unannotated_segment_exit : Norm(-1, 0.1) , end_to_unannotated_segment : Norm(-9, 0.1) , 
    end_to_right_flank : Norm(0, 0.1) , end_to_terminal : Norm(0, 0.1)
    },
 flank_init=Const(0.0),
 prior=ProfileHMMTransitionPrior(match_comp=1, insert_comp=1, delete_comp=1, alpha_flank=7000, alpha_single=100000000

/home/beckerf/mambaforge/envs/learnMSAdev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-10 10:06:47.522010: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Computing all embeddings (this may take a while).
Fitting models of lengths [139 134 128 133 141 135 134 135 141 129] on 17976 sequences.
Batch size= 32 Learning rate= 0.05
Using sequence weights  [0.00025549 0.00025549 0.00025549 ... 0.00105597 0.00485437 0.00019759] .
Using 1 GPUs.


2024-01-10 10:23:13.664117: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-10 10:23:13.664375: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-10 10:23:13.664556: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-10 10:23:13.664770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-10 10:23:13.664948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Epoch 1/10


2024-01-10 10:23:17.285629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-10 10:23:17.285887: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-10 10:23:17.286068: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-10 10:23:17.286282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-10 10:23:17.286461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

418/418 - 145s - loss: 431.1493 - loglik: -4.4005e+02 - logprior: 9.2473 - 145s/epoch - 346ms/step
Epoch 2/10
418/418 - 122s - loss: 358.8813 - loglik: -4.2947e+02 - logprior: 70.9783 - 122s/epoch - 292ms/step
Epoch 3/10
418/418 - 131s - loss: 339.8258 - loglik: -4.2771e+02 - logprior: 88.2706 - 131s/epoch - 313ms/step
Epoch 4/10
418/418 - 129s - loss: 335.3857 - loglik: -4.2849e+02 - logprior: 93.4867 - 129s/epoch - 308ms/step
Epoch 5/10


2024-01-10 10:33:54.691395: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.63GiB (rounded to 4972953600)requested by op gradient_tape/model_5/msa_hmm_layer/mvn_emitter/MatVec/MatMul/MatMul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-01-10 10:33:54.691487: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2024-01-10 10:33:54.691500: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 705, Chunks in use: 703. 176.2KiB allocated for chunks. 175.8KiB in use in bin. 4.2KiB client-requested in use in bin.
2024-01-10 10:33:54.691509: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 520, Chunks in use: 520. 367.8KiB allocated for chunks. 367.8KiB in use in bin. 2

Out of memory. A resource was exhausted.
Try reducing the batch size (-b). The current batch size was: functools.partial(<function get_adaptive_batch_size_with_language_model at 0x7f15d5f73910>, embedding_dim=32).


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [7]:
!id_list=$(sed -n '/^>/p' {ref_filename} | sed 's/^.//') ; export MAX_N_PID_4_TCOFFEE=10000000 ; t_coffee -other_pg seq_reformat -in test/data/interactive.alignment.fasta -action +extract_seq_list ${{id_list[@]}} +rm_gap > test/data/interactive.projection.fasta

HERE: 1hna
HERE: 5gsta
HERE: 1gsua
HERE: 1gta
HERE: 2fhea
HERE: 1fhe
HERE: 2gsra
HERE: 1glpa
HERE: 17gsa
HERE: 1guha
HERE: 1guka
HERE: 1gula
HERE: 1gtua
HERE: 3gtub


In [8]:
!t_coffee -other_pg aln_compare -al1 {ref_filename} -al2 test/data/interactive.projection.fasta -compare_mode sp

*****************************************************
seq1       seq2          Sim   [ALL]           Tot  
gluts         14         39.4    87.7 [100.0]   [38000]


In [7]:
alignment_model.model.layers[-3].cell.emitter[0].mvn_mixture.component_scales()[0,5,0]

<tf.Tensor: shape=(32, 32), dtype=float32, numpy=
array([[0.24569795, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.22047585, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.17972693, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.22577955, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.20833336,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.20910044]], dtype=float32)>

In [ ]:
alignment_model.model.layers[-3].cell.emitter[0].mvn_mixture.component_scales()[0,5,0]

<tf.Tensor: shape=(32, 32), dtype=float32, numpy=
array([[0.3066573 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.2782405 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.24676737, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.27079624, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.2663671 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.25026017]], dtype=float32)>